# Spatiotemporal analysis of soil moisture 



* **Special requirements:** A Google account, access to Google Earth Engine; 
* **Prerequisites:** Knowledge of microwave remote sensing and basic statistics


## Background
**Microwave remote sensing** provides a unique capability for direct observation of soil moisture. The microwave portion of the spectrum covers the range from approximately 1cm to 1m in wavelength. The longer wavelength microwave radiation can penetrate through cloud cover, haze and dust and allows detection of microwave energy under almost all weather and enviornmental conditions. 

<img align="centre" src=../figures/7.1_fig1.PNG width="800">

The **satellite soil moisture** product we are going to use in this exerise is the NASA-USDA Enhanced **SMAP(Soil Moisture Active and Passive)** Global Soil Moisture Data. SMAP is a NASA environmental monitoring satellite launched on 31 January 2015. Soil moisture content can be mapped via the radiometer data at a spatial resolution of 36 km every 2-3 days. You can read more about the mission here https://smap.jpl.nasa.gov/

The **estimated soil moisture** we are going to use is from the **GLDAS (Global Land Data Assimilation System)**. GLDAS ingests satellite- and ground-based observations using advanced land surface modelling and data assimilation techniques to generate optimal fields of land surface states and fluxes. You can read more about the system here https://ldas.gsfc.nasa.gov/gldas


## Aims of the practical session

1. Understand the spatial and temporal resolution of satellite soil moisture observations
2. Understand the spatial and temporal resolution of soil moisture estimates from land surface model
3. Compare soil moisture observations from satellite and land surface model


## Description

1. First we visualise surface soil moisture observed by SMAP satellite for the 2022 Flood in Feb 
2. Then we compare the soil moisture estimates from land surface model 
3. Finally extract the soil moisture time series at a point from the land surface model and calculate the correlation with SMAP observations

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Scripts` directory.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geemap as gmap
import ee
import geemap.colormaps as cm

### Connect to Google Earth Engine (GEE)

Connect to the GEE so we can access GEE datasets and computing assets.
You may be required to input your Google account name and password. Please keep those safe and don't share them with anyone.

## Visualise satellite soil moisture map for eastern Australia floods in 2022

Let's look at soil moisture map in February 2022 for the recent eastern Australia floods. Note that the product we used is a level3 data and the moisture content is in mm unit. Details of the product can be found [here](https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V021_NOAH_G025_T3H#description). 

In [ ]:
# Create an interactive map. 
Map = gmap.Map(center=[-31,150], zoom=5)

# Tell GEE which dataset we want, and select a data layer within the dataset.
# Here we're selecting the 'ssm' layer from the dataset for the surface soil moisture.
ssm = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture').filterDate('2022-02-12',"2022-03-01").select('ssm');

# Then we tell GEE how we want to visualize the data.

# here we use the colormaps from the geemap
palette = cm.palettes.gist_earth_r
vis_params = {
#     We give minimum and maximum values,
  'min':0,
  'max': 28,
    
#   And we select the collor palette  
  'palette': palette
};

# we add the surface soil moisture data as a new 'layer' in our map.
Map.addLayer(ssm, vis_params, 'surface soil moisture', True, 1)

# here we want to include the colorbar to interpretate the values
colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']
Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="surface soil moisture")
Map

## Create a time slider to visualize the change in soil moisture

In [ ]:
# firstly we need to convert image collection to image for time series
SSMimage = ssm.toBands()

# you can get see the information of each band
SSMimage.getInfo()

In [ ]:
# to visualise the soil moisture change within the two weeks time, we create a time slider here
# you can pause to see the map for each date
Map.add_time_slider(SSMimage, vis_params)
Map

## Exercise 1 Understand satellite soil moisture

1. What is the temporal resolution of this product? 

2. Can you tell the rough start date (wettest date) of the floods around Brisbane? You can plot the soil moisture values for some points near Brisbane using the plotting tool.

3. Can you extract the soil moisture values for a point (152.75,-27.15) for this two-week period and compare the it with 12-28 Feb 2019?

4. What can you tell from the two time series?

In [ ]:
# can you add the map of start date you found as a new layer?
Map.addLayer(SSMimage.select(4), vis_params, 'Flood start date', True, 1)
Map


In [ ]:
# Your Code Here. You can follow the steps below

# Firstly let's create a point

point = ee.Geometry.Point(152.75,-27.15)

Map.addLayer(point,{'color':'red'},'point')
Map

In [ ]:
# let's save the end date of each three-day composite in a list
numbands = len(SSMimage.getInfo().get('bands')) # number of bands
bandid  = [SSMimage.getInfo().get('bands')[di].get('id') for di in range(numbands)]
dates = [id.split('_')[4] for id in bandid] #get split the string with '_' and get the enddate
dates

In [ ]:
# extract the soil moisture values for the selected point and save it in a dataframe
ssm2022 = SSMimage.sampleRegions(point,geometries=True)
df_22 = gmap.ee_to_pandas(ssm2022,col_names=bandid)
df_22

In [ ]:
# plot the time series of the soil moisture value of this point
plt.plot(dates,df_22.values[0],'-o',label='ssm 2022')
plt.ylabel('surface soil moisture (mm)')
plt.legend()

In [ ]:
# NOW, can you extract the same period in 2020 for the same point and plot the two time series together?

# your code here
# Here we're selecting the 'ssm' layer from the dataset for the surface soil moisture.
ssm19 = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture').filterDate('2019-02-12',"2019-03-01").select('ssm');
SSMimage19 = ssm19.toBands()
ssm2019 = SSMimage19.sampleRegions(point,geometries=True)
numbands = len(SSMimage19.getInfo().get('bands')) # number of bands
bandid_19  = [SSMimage19.getInfo().get('bands')[di].get('id') for di in range(numbands)]
df_19 = gmap.ee_to_pandas(ssm2019,col_names=bandid_19)
df_19

In [ ]:
# plot the two time series together, can you see the difference?

plt.plot(df_22.values[0],'-o',label='ssm 2022')
plt.plot(df_19.values[0],'-o',label='ssm 2019')
plt.ylabel('surface soil moisture (mm)')
plt.legend()

## Compare model estimated surface soil moisture with satellite soil moisture
let's look at the surface soil moisture estimates from the NOAH model from the GLDAS. The details of this product can be found [here](https://developers.google.com/earth-engine/datasets/catalog/NASA_GLDAS_V021_NOAH_G025_T3H#description)


In [ ]:
# let's create a map to visualise the model estimates 

Map2 = gmap.Map(center=[-31,150], zoom=5)

# Here we're selecting the 'SoilMoi0_10cm_inst' from the dataset for the surface soil moisture at 0-10cm.
gldas = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filterDate('2022-02-24',"2022-02-25").select('SoilMoi0_10cm_inst');

# Then we tell GEE how we want to visualize the data.

# here we use the colormaps from the geemap
palette = cm.palettes.gist_earth_r
vis_params = {
#     We give minimum and maximum values,
  'min':2,
  'max': 48,
    
#   And we select the collor palette  
  'palette': palette
};

# we add the modelled surface soil moisture data as a new 'layer' in our map.
Map2.addLayer(gldas, vis_params, 'SoilMoi0_10cm_inst', True, 1)

# here we want to include the colorbar to interpretate the values
colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']
Map2.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="SoilMoi0_10cm_inst")


# to view the series of images, we create the time slider again 
GLDASimages = gldas.toBands()
Map2.add_time_slider(GLDASimages, vis_params)
Map2

## Exersice 2 Compare model estimated surface soil moisture with satellite soil moisture

1. What is the temporal resolution of this model outputs? Which product has better spatial details?

2. Can you visualise the moisture change within one day for the same point (152.75,-27.15)? When was the moisture content the highest?

3. Can you calculate the daily average soil moisture map for 24 Feb 2022? What is the difference to the satellite product?

In [ ]:
# Your Code Here

# check the bands id for the temporal resolution
numbands = len(GLDASimages.getInfo().get('bands')) # number of bands
bandid  = [GLDASimages.getInfo().get('bands')[di].get('id') for di in range(numbands)]
bandid

In [ ]:
# Your Code Here

# extract the soil moisture values for the point and save it in a dataframe

gldas22 = GLDASimages.sampleRegions(point,geometries=True)
df_gldas = gmap.ee_to_pandas(gldas22,col_names=bandid)
df_gldas

In [ ]:
# Your Code Here

# plot the subdaily variation of soil moisture 

# get the time of each band 
times = [di.split('_')[1] for di in bandid]

# Can you get the average value of moisture at this point? 

meansm = df_gldas.mean(axis=1)

# plot the soil moisture and show the average value in the title
plt.figure(figsize=(8,3))
plt.plot(times,df_gldas.values[0],'-s',label='GLDAS')
plt.ylabel('model soil moisture')
plt.legend()
plt.title('The average soil moisture for 24-02-2022 at (-27.15,152.75) is %3.2f mm' %meansm.values)

In [ ]:
# Your code Here

# Now visualise the average map of soil moisture for Australia?
Map3 = gmap.Map(center=[-31,150], zoom=5)

# Here we're selecting the 'SoilMoi0_10cm_inst' from the dataset for the surface soil moisture at 0-10cm.
gldas_avg = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filterDate('2022-02-24',"2022-02-25").select('SoilMoi0_10cm_inst').mean();

# Then we tell GEE how we want to visualize the data.

# here we use the colormaps from the geemap
palette = cm.palettes.gist_earth_r
vis_params = {
#     We give minimum and maximum values,
  'min':2,
  'max': 48,
    
#   And we select the collor palette  
  'palette': palette
};

# we add the modelled surface soil moisture data as a new 'layer' in our map.
Map3.addLayer(gldas_avg, vis_params, 'Daily_Average_SoilMoi0_10cm_inst', True, 1)

# here we want to include the colorbar to interpretate the values
colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']
Map3.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="Daily_Average_SoilMoi0_10cm_inst")
Map3

In [ ]:
# now let's creat a split map to compare two products

vis_params = {
#     We give minimum and maximum values,
  'min':0,
  'max': 48,
    
#   And we select the collor palette  
  'palette': cm.palettes.gist_earth_r
};
# create the left and right layer map for display
left_layer = gmap.ee_tile_layer(gldas_avg,vis_params,'Daily_Average_SoilMoi0_10cm_inst')

vis_params = {
#     We give minimum and maximum values,
  'min':0,
  'max': 48,
    
#   And we select the collor palette  
  'palette': cm.palettes.gist_earth_r
};

right_layer = gmap.ee_tile_layer(SSMimage.select(4),vis_params,'Satellite_observed_soil_moisture_24022022')
Map4 = gmap.Map(center=[-30,145], zoom=4)
Map4.split_map(left_layer,right_layer)
Map4.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="Daily_Average_SoilMoi0_10cm_inst")
Map4

## Statistical analysis

Let's calculate the root-mean-square error (RMSE) and the correlation between the model estimates and observations. 


Root-mean-square eroor (RMSE) is a frequently used measure of the differences between values predicted by a model and the values observed. When measuring the average difference between two time series $x, y$, the formular is

$ RMSE = \sqrt{\frac{\sum \limits _{t=1}^{T}(x-y)^2}{T}}$

Pearson correlation coeffcient (*r*) is a measure of linear correlation between two sets of data. It is the ratio between the covariance of two variables and the product of their standard deviations

$ r = \frac{cov(x,y)}{\sigma_x\sigma_y}$


## Exercise 3 time series analysis

1. Extract the modelled soil moisture and satellite soil moisture for the same point for a three-year period from 2019-2021

2. Can you resample the modelled 3-hourly data to 3-day data to be consistent with SMAP observations? Plot the resampled data

2. What is the RMSE and correlation between model outputs and SMAP observations?

In [ ]:
# Now let's extract the soil moisture values for a three year period from 2019-2021 for the given point

# Here we're selecting the 'SoilMoi0_10cm_inst' from the dataset for the surface soil moisture at 0-10cm.
startDate = ee.Date('2019-01-01')
endDate = ee.Date('2021-12-31')
gldas = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filterDate(startDate,endDate).select('SoilMoi0_10cm_inst')

# here we define a function to extract the value for a region or a point
def poi_extract(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=point).get('SoilMoi0_10cm_inst')
    return img.set('date', img.date().format()).set('model_ssm',mean)

poi_reduced_imgs = gldas.map(poi_extract)

# create a dataframe for the soil moisture values
nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','model_ssm']).values().get(0)
df_mod = pd.DataFrame(nested_list.getInfo(), columns=['date','model_ssm'])
df_mod

In [ ]:
# Can you save the values of the SMAP product at this point to a dataframe here? 

# Your Code Here

smap = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture').filterDate(startDate,endDate).select('ssm')

def poi_extract(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=point).get('ssm')
    return img.set('date', img.date().format()).set('obs_ssm',mean)

poi_reduced_imgs = smap.map(poi_extract)

# create a dataframe for the soil moisture values
nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','obs_ssm']).values().get(0)
df_smap = pd.DataFrame(nested_list.getInfo(), columns=['date','obs_ssm'])
df_smap

In [ ]:
# since the SMAP soil moisture observation is available every three day, we need to resample the 3-hourly outputs from the model to match SMAP observations
df_mod['date'] = pd.to_datetime(df_mod['date']) #the date in the dataframe is RangeIndex, and need to be converted to datetimeindex
df_smap['date'] = pd.to_datetime(df_smap['date'])
df_mod_1d = df_mod.resample('1D',on='date').mean() #because the start date of a three-day period is not from the first day of the year, we get daily average first
df_mod_3d = df_mod_1d.reindex(df_smap['date'].dt.date) #then reindex to the SMAP date
df_mod_3d['obs_ssm'] = df_smap['obs_ssm'].values #append the SMAP observations in the same dataframe
df_mod_3d

In [ ]:
# let's plot the two time series
df_mod_3d.plot(figsize=(10,4))

### Do you think the model simulations agree with the observations? Can you calculate the RMSE and correlation coefficient?

calculate the RMSE and pearson r 


In [ ]:
# RMSE calculation 

# Your code here

rmse = ((df_mod_3d.model_ssm-df_mod_3d.obs_ssm)**2).mean()**.5
rmse

In [ ]:
# Pearson r calculation 

# Your code here

r = df_mod_3d.corr(method='pearson') # you can call the function 

# or write your own

r = (((df_mod_3d.model_ssm-df_mod_3d.model_ssm.mean())*(df_mod_3d.obs_ssm-df_mod_3d.obs_ssm.mean())).sum()/(len(df_mod_3d)-1))/(df_mod_3d.model_ssm.std()*df_mod_3d.obs_ssm.std())
r

## References

This is where the references go. For exmaple:
- Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305

***

## Additional information

**License:** The code in this notebook is prepared by Siyuan Tian. The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle (**check**) site or contact Siyuan (siyuan.tian@anu.edu.au)

**Last modified:** July 2022